In [28]:
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option("display.max_columns",None)
pd.set_option("display.float_format",lambda x: '%.5f' % x)

df_ = pd.read_excel("online_retail_II.xlsx",sheet_name="Year 2009-2010")
df_.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [29]:
df = df_.copy()

In [30]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [31]:
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C",na=False)]

In [32]:
df = df[(df["Quantity"]>0)]
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,407695.00000,407695,407695.00000,407695.00000
mean,13.58669,2010-07-01 10:10:10.782177792,3.29419,15368.50411
min,1.00000,2009-12-01 07:45:00,0.00000,12346.00000
25%,2.00000,2010-03-26 14:01:00,1.25000,13997.00000
50%,5.00000,2010-07-09 15:46:00,1.95000,15321.00000
75%,12.00000,2010-10-14 17:09:00,3.75000,16812.00000
max,19152.00000,2010-12-09 20:01:00,10953.50000,18287.00000
std,96.84223,NaN,34.75666,1679.79570


In [33]:
df["total_price"] = df["Quantity"] * df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,total_price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000


In [34]:
df.groupby("Customer ID").agg({"Invoice":lambda x:len(x)})

,Invoice
Customer ID,
12346.00000,33
12347.00000,71
12348.00000,20
12349.00000,102
12351.00000,21
...,...
18283.00000,230
18284.00000,28
18285.00000,12


In [35]:
cltv_c=df.groupby("Customer ID").agg({"Invoice":lambda x:x.nunique(),
                               "Quantity":lambda x:x.sum(),
                               "total_price":lambda x:x.sum()});cltv_c

,Invoice,Quantity,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000
...,...,...,...
18283.00000,6,336,641.77000
18284.00000,1,494,461.68000
18285.00000,1,145,427.00000


In [36]:
cltv_c.rename(columns ={"Invoice":"a"})

,a,Quantity,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000
...,...,...,...
18283.00000,6,336,641.77000
18284.00000,1,494,461.68000
18285.00000,1,145,427.00000


In [37]:
cltv_c.columns = ["total_transaction","total_unit","total_price"]
cltv_c.index = cltv_c.index.astype(int)
cltv_c

,total_transaction,total_unit,total_price
Customer ID,,,
12346,11,70,372.86000
12347,2,828,1323.32000
12348,1,373,222.16000
12349,3,993,2671.14000
12351,1,261,300.93000
...,...,...,...
18283,6,336,641.77000
18284,1,494,461.68000
18285,1,145,427.00000


ortalama harcanan para: toplam harcanan para / toplam yapılan işlem sayısı

In [38]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]

purchase frequency = (total_transaction / total_number_of_customers)

In [39]:
cltv_c.shape # satırları eşsiz müşteriler

(4314, 4)

In [40]:
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency
Customer ID,,,,,
12346,11,70,372.86000,33.89636,0.00255
12347,2,828,1323.32000,661.66000,0.00046
12348,1,373,222.16000,222.16000,0.00023
12349,3,993,2671.14000,890.38000,0.00070
12351,1,261,300.93000,300.93000,0.00023
...,...,...,...,...,...
18283,6,336,641.77000,106.96167,0.00139
18284,1,494,461.68000,461.68000,0.00023
18285,1,145,427.00000,427.00000,0.00023


repeat_rate = birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler

In [47]:
repeat_rate=cltv_c[cltv_c["total_transaction"]>1].shape[0] / cltv_c.shape[0]
churn_rate = 1-repeat_rate
churn_rate 

0.3293926750115902

profit margin = total_price * 0.10 (şirket belirliyo)

In [49]:
cltv_c["profit_margin"]= cltv_c["total_price"]*0.10

customer_value = average_order_value * purchase_frequency

In [51]:
cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]

cltv = (customer_value/churn_rate) * profit_margin

In [53]:
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
12346,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357
12347,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546
12348,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326
12349,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041
12351,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290
...,...,...,...,...,...,...,...,...
18283,6,336,641.77000,106.96167,0.00139,64.17700,0.14876,28.98443
18284,1,494,461.68000,461.68000,0.00023,46.16800,0.10702,14.99989
18285,1,145,427.00000,427.00000,0.00023,42.70000,0.09898,12.83103


In [54]:
cltv_c.sort_values(by="cltv",ascending=False)

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003
...,...,...,...,...,...,...,...,...
15913,1,3,6.30000,6.30000,0.00023,0.63000,0.00146,0.00279
13788,1,1,3.75000,3.75000,0.00023,0.37500,0.00087,0.00099
14095,1,1,2.95000,2.95000,0.00023,0.29500,0.00068,0.00061


In [55]:
cltv_c.describe()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
count,4314.00000,4314.00000,4314.00000,4314.00000,4314.00000,4314.00000,4314.00000,4314.00000
mean,4.45410,1284.01113,2047.28866,378.14723,0.00103,204.72887,0.47457,5883.60651
std,8.16866,6458.45205,8912.52324,492.51721,0.00189,891.25232,2.06595,156068.63594
min,1.00000,1.00000,0.00000,0.00000,0.00023,0.00000,0.00000,0.00000
25%,1.00000,158.00000,307.95000,181.95632,0.00023,30.79500,0.07138,6.67369
50%,2.00000,382.00000,705.55000,286.94600,0.00046,70.55500,0.16355,35.03173
75%,5.00000,995.25000,1722.80250,423.53375,0.00116,172.28025,0.39935,208.87042
max,205.00000,220600.00000,349164.35000,11880.84000,0.04752,34916.43500,80.93749,8579573.77276


In [56]:
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
12346,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357
12347,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546
12348,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326
12349,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041
12351,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290
...,...,...,...,...,...,...,...,...
18283,6,336,641.77000,106.96167,0.00139,64.17700,0.14876,28.98443
18284,1,494,461.68000,461.68000,0.00023,46.16800,0.10702,14.99989
18285,1,145,427.00000,427.00000,0.00023,42.70000,0.09898,12.83103


In [64]:
cltv_c["segment"] = pd.qcut(cltv_c["cltv"],5,labels=["D","C","B","A","S"])
cltv_c.sort_values(by="cltv",ascending=False)

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
18102,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276,S
14646,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829,S
14156,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615,S
14911,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978,S
13694,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003,S
...,...,...,...,...,...,...,...,...,...
15913,1,3,6.30000,6.30000,0.00023,0.63000,0.00146,0.00279,D
13788,1,1,3.75000,3.75000,0.00023,0.37500,0.00087,0.00099,D
14095,1,1,2.95000,2.95000,0.00023,0.29500,0.00068,0.00061,D


In [65]:
cltv_c.groupby("segment").agg(["count","sum","mean","max","min"])

total_transaction                          total_unit           \
                    count    sum     mean  max min      count      sum   
segment                                                                  
D                     863   1016  1.17729    9   1        863    82034   
C                     863   1438  1.66628   11   1        863   184472   
B                     862   2306  2.67517    9   1        862   371160   
A                     863   3948  4.57474   62   1        863   758142   
S                     863  10507 12.17497  205   1        863  4143416   

                               total_price                           \
              mean     max min       count           sum       mean   
segment                                                               
D         95.05678     630   1         863  131305.71000  152.15030   
C        213.75666    1500   1         863  316013.92100  366.18067   
B        430.58005    5694   2         862  620163.79200  719.44755   
A        878.49594    6500   1         863 1275073.74000 1477.48985   
S       4801.17729  220600   1         863 6489446.11100 7519.63628   

                                average_order_value                         \
                 max        min               count          sum      mean   
segment                                                                      
D          251.71000    0.00000                 863 119766.32097 138.77905   
C          505.80000  252.48000                 863 232074.95962 268.91652   
B          990.00000  506.63000                 862 291609.02249 338.29353   
A         2145.06000  991.79000                 863 369634.21995 428.31312   
S       349164.35000 2146.38000                 863 618242.64473 716.38777   

                              purchase_frequency                          \
                max       min              count     sum    mean     max   
segment                                                                    
D         251.10000   0.00000                863 0.23551 0.00027 0.00209   
C         502.52000  33.89636                863 0.33333 0.00039 0.00255   
B         988.04000  63.68750                862 0.53454 0.00062 0.00209   
A        2135.46000  27.40484                863 0.91516 0.00106 0.01437   
S       11880.84000 130.85778                863 2.43556 0.00282 0.04752   

                profit_margin                                               \
            min         count          sum      mean         max       min   
segment                                                                      
D       0.00023           863  13130.57100  15.21503    25.17100   0.00000   
C       0.00023           863  31601.39210  36.61807    50.58000  25.24800   
B       0.00023           862  62016.37920  71.94476    99.00000  50.66300   
A       0.00023           863 127507.37400 147.74898   214.50600  99.17900   
S       0.00023           863 648944.61110 751.96363 34916.43500 214.63800   

        customer_value                                      cltv  \
                 count        sum    mean      max     min count   
segment                                                            
D                  863   30.43711 0.03527  0.05835 0.00000   863   
C                  863   73.25311 0.08488  0.11725 0.05853   863   
B                  862  143.75609 0.16677  0.22949 0.11744   862   
A                  863  295.56647 0.34249  0.49723 0.22990   863   
S                  863 1504.27587 1.74308 80.93749 0.49754   863   

                                                            
                   sum        mean           max       min  
segment                                                     
D           1634.24168     1.89368       4.45869   0.00000  
C           8442.87553     9.78317      18.00377   4.48601  
B          32591.15043    37.80876      68.97255  18.06291  
A         138987.51485   161.05158     323.80594  69.22220  
S       2520022

In [89]:
df = df_.copy()

In [90]:
def create_cltv_c(df,profit=0.10):
    df.dropna(inplace=True)
    df = df[~df["Invoice"].str.contains("C",na=False)]
    df = df[df["Quantity"]>0]
    df["total_price"] = df["Quantity"] * df["Price"]
    cltv_c = df.groupby("Customer ID").agg({"Invoice":lambda x:x.nunique(),
                                            "Quantity":lambda x:x.sum(),
                                            "total_price":lambda x:x.sum()})
    cltv_c.index = cltv_c.index.astype(int)
    cltv_c.columns = ["total_transaction","total_unit","total_price"]
    
    #avg_order_value
    cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]
    
    #purchase_frequency
    cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]
    
    #repeat_rate & churn_rate
    repeat_rate = cltv_c[cltv_c["total_transaction"]>1].shape[0] / cltv_c.shape[0]
    churn_rate = 1 - repeat_rate
    
    #profit_marigin
    cltv_c["profit_margin"] = cltv_c["total_price"] * profit
    
    #customer_value
    cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]
    
    #customer_lifetime_value
    cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]
    
    #segment
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"],q=4,labels=["D","C","B","A"])
    
    return cltv_c

In [92]:
clv=create_cltv_c(df)

In [93]:
clv

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
12346,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357,C
12347,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546,B
12348,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326,D
12349,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041,A
12351,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290,D
...,...,...,...,...,...,...,...,...,...
18283,6,336,641.77000,106.96167,0.00139,64.17700,0.14876,28.98443,C
18284,1,494,461.68000,461.68000,0.00023,46.16800,0.10702,14.99989,C
18285,1,145,427.00000,427.00000,0.00023,42.70000,0.09898,12.83103,C


In [94]:
1200 /150

8.0